In [19]:
import pandas as pd
import json
import numpy as np
import networkx as nx


with open('data.json', 'r', encoding='utf-8') as json_data:
    data = json.load(json_data)

normalized_data = []

for key, value in data.items():
    user_id = key
    info = value.get('info', [{}])[0]  
    groups = value.get('groups', [])
    if not groups:
        record = {**info, 'id': user_id}
        normalized_data.append(record)
        continue
    for group in groups:
        record = {**info, **group, 'id': user_id, 'group_id': group['id']}
        normalized_data.append(record)

df = pd.DataFrame(normalized_data)

In [20]:
df.head()

,id,first_name,last_name,can_access_closed,is_closed,deactivated,name,screen_name,type,photo_50,photo_100,photo_200,group_id
0,114668685,Alexey,Dozorov,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64172662,Alexander,Antonnikov,True,False,deleted,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,256740,Mikhail,Podgorny,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,118465737,Renat,Khakimov,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90084913,Anna,Solovskaya,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df = df.drop(['photo_50', 'photo_100', 'photo_200', 'deactivated', 'can_access_closed', 'is_closed', 'type'], axis=1)

In [22]:
df

,id,first_name,last_name,name,screen_name,group_id
0,114668685,Alexey,Dozorov,NaN,NaN,NaN
1,64172662,Alexander,Antonnikov,NaN,NaN,NaN
2,256740,Mikhail,Podgorny,NaN,NaN,NaN
3,118465737,Renat,Khakimov,NaN,NaN,NaN
4,90084913,Anna,Solovskaya,NaN,NaN,NaN
...,...,...,...,...,...,...
365358,794082984,Marina,Manerova,Путь к себе. Эзотерика.,prosnis24,46558304.0
365359,794082984,Marina,Manerova,Новости | Саранск City | Republic of Mordovia,gorod_saransk,36809318.0
365360,794082984,Marina,Manerova,САРАНСК Онлайн,rmsar,60178575.0
365361,794082984,Marina,Manerova,10 канал,10tvrm,70336234.0


In [23]:
df['group_id'] = df['group_id'].astype(float).astype('Int64')

In [24]:
df.rename(columns={'name':'group_name', 'screen_name': 'group_screen_name', 'id': 'user_id', 'first_name': 'user_first_name', 'last_name': 'user_last_name'}, inplace=True)

In [25]:
df.dropna(how='any', inplace=True)

In [26]:
df

,user_id,user_first_name,user_last_name,group_name,group_screen_name,group_id
7,125402825,Artyom,Amosov,Sperry Ukraine,sperrytopsiderukraine,19603458
8,125402825,Artyom,Amosov,Dubstep set up,dubstepforum,22778924
9,125402825,Artyom,Amosov,КИНОМАНИЯ | Фильмы и сериалы,kino_mania,22798006
10,125402825,Artyom,Amosov,MDK,mudachyo,23148107
11,125402825,Artyom,Amosov,Artist Music,artistmusic,23162459
...,...,...,...,...,...,...
365358,794082984,Marina,Manerova,Путь к себе. Эзотерика.,prosnis24,46558304
365359,794082984,Marina,Manerova,Новости | Саранск City | Republic of Mordovia,gorod_saransk,36809318
365360,794082984,Marina,Manerova,САРАНСК Онлайн,rmsar,60178575
365361,794082984,Marina,Manerova,10 канал,10tvrm,70336234


In [27]:
G = nx.Graph()         # создаем пустой граф

In [28]:
edgelist = df[['user_id', 'group_id']]

In [29]:
edgelist

,user_id,group_id
7,125402825,19603458
8,125402825,22778924
9,125402825,22798006
10,125402825,23148107
11,125402825,23162459
...,...,...
365358,794082984,46558304
365359,794082984,36809318
365360,794082984,60178575
365361,794082984,70336234


In [30]:
G1=nx.from_pandas_edgelist(edgelist, 'user_id', 'group_id', create_using=nx.Graph())

# Центральность по посредничеству
*Проблема в том, что временная сложность алгоритма равняется O(VE), что гарантирует гигантское время прогона. Но и на случайную выборку тоже сложно надеяться, потому что у всех людей различные группы. Но что-то общее все-таки присутствует*

**В алгоритме:** `k=100` **- случайные узлы графа**

In [45]:

betweenness_centrality = sorted(list(nx.betweenness_centrality(G1, k=100).items()), key=lambda i: i[1], reverse=True)

In [49]:
betweenness_centrality[:10]
# closeness_centrality

[(91412129, 0.16387765544179006),
 (39325472, 0.07279161350350567),
 (28905875, 0.05477534978921373),
 (29534144, 0.03324563830554355),
 (66678575, 0.03109110053158036),
 (45745333, 0.030560138312570945),
 (29573241, 0.028636152732646688),
 (91050183, 0.027953288489965575),
 (72081844, 0.02673549284651425),
 (38801259, 0.02470868464050533)]

In [50]:
group_values_df = pd.DataFrame(betweenness_centrality, columns=['group_id', 'weight'])
merged_df = pd.merge(group_values_df, df, on='group_id', how='left')
result = [(row['group_id'], row['weight'], row['group_name']) for idx, row in merged_df.iterrows()]
result_df = pd.DataFrame(result, columns=['group_id', 'weight', 'group_name'])

*Центральность по посредничеству - результат*

In [51]:
result_df.drop_duplicates()

,group_id,weight,group_name
0,91412129,0.163878,Подслушано Рузаевка (САРАНСК)
2555,39325472,0.072792,Типичный Балаково!
4459,28905875,0.054775,Рифмы и Панчи
5993,29534144,0.033246,Лентач
6453,66678575,0.031091,"Овсянка, сэр!"
...,...,...,...
340986,198285669,0.000000,Нейрографика. Психолог. Дарья Петровичева
340987,217306195,0.000000,Служба Семьи в Белорецком районе
340988,198725806,0.000000,"Живые открытки, видео."
340989,213430879,0.000000,Жильё для отдыха Дивноморск


In [35]:
# Центральность по собственному вектору
eigenvector_centrality = sorted(list(nx.eigenvector_centrality_numpy(G1).items()), key=lambda i: i[1], reverse=True)

AmbiguousSolution: `eigenvector_centrality_numpy` does not give consistent results for disconnected graphs

In [13]:
# Центральность по близости
closeness_centrality = sorted(list(nx.closeness_centrality(G1).items()), key=lambda i: i[1], reverse=True)

KeyboardInterrupt: 